# Obtain valid set of reactions 
Required Python code to load pertinent modules.

In [ ]:
%run "core.ipynb"
analysis = load_analysis()

In [ ]:
modelDB = thermo.model.Database()

## Rxn boundary from Berman (1988)
### Spl + 2En = Fo + Py
![](rxn_bound_berman1988.png)

# Create rxn for nearly pure phases

In [ ]:
phase_symbols = ['SplS','Opx','Ol','Grt']
endmember_ids = [3, 1, 5, 2]

mols = {}

for sym, endmem_id in zip(phase_symbols, endmember_ids):
    iphs = modelDB.get_phase(sym)
    imol = 1e-3*np.ones(iphs.endmember_num)
    imol[endmem_id] = 1.0
    mols[sym] = imol
    print(iphs.endmember_names[endmem_id])

In [ ]:
mols

In [ ]:
rxn_coefs = np.array([-7.0, -20, 7, 20])
rxn_coefs /= 0.5*np.sum(np.abs(rxn_coefs))
print(rxn_coefs)

In [ ]:
rxn_pyr = modelDB.get_rxn(phase_symbols, endmember_ids, rxn_coefs, 
                          coefs_per_atom=True)


In [ ]:
rxn_pyr.affinity(1373.0, 18500.0, mols=mols)

In [ ]:
Tbnd_ref = 1373.0
Pbnd_ref = rxn_pyr.boundary(T=Tbnd_ref, mols=mols, init_guess=18500.0)[0]

In [ ]:
print(Tbnd_ref, ' [C] , ', Pbnd_ref, ' [bars]')

In [ ]:
Trxn_bnd = np.linspace(900,2000,31)
Prxn_bnd = np.zeros(Trxn_bnd.shape)

for ind, iT in enumerate(Trxn_bnd):
    Prxn_bnd[ind] = rxn_pyr.boundary(T=iT, mols=mols, init_guess=18000.0)

In [ ]:
plt.figure()
plt.plot(Trxn_bnd-273, Prxn_bnd/1e3, 'k-')
plt.ylim(10,35)

plt.xlabel('T (C)')
plt.ylabel('P (kbar)')